<a href="https://colab.research.google.com/github/KSeungBin/ros_deeplearning/blob/master/08_yolo_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# darknet 모델 github에서 다운받기

In [ ]:
!git clone https://github.com/AlexeyAB/darknet
%cd darknet
# compile option 변경(openCV, GPU, DNN 사용하겠다)
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

!make # compile : darknet 디렉토리 하위에 darknet 실행파일이 생성되었는지 확인

In [ ]:
#transfer learning 전이학습 : 전이학습(scratch learning, transfer learning) 중 transfer learning하려면 훈련(학습)된 weight file 필요
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

In [ ]:
%matplotlib inline
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  #확대시 공백을 채울때 방법 채텍 : interpolation cubic 주변의 color를 참조하여 보간
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf() # get current figure  출력종이
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

def upload():
  from google.colab import files
  uploaded = files.upload()
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

def download(path):
  from google.colab import files
  files.download(path) 

In [ ]:
# parameters(arguments) 중요!
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/person.jpg
# ./darknet : 현재 디렉토리(darknet)의 darknet 실행 파일 launch
# detector test : 하나의 이미지를 대상으로 함 (동영상이나 카메라 촬영 영상을 대상으로 하면 detector demo)
# cfg/coco.data : 환경설정 file (cfg/coco.names에 설정되어 있는 class 갯수를 갖고 있음)
# cfg/yolov4.cfg : 현재 진행되는 yolo가 사용하는 network : [convolutional]에 지정된대로 yolo실행 시 configuration 진행

In [ ]:
# imShow 또는 메뉴바의 predictions.jpg를 더블 클릭해서 weight file이 이미 학습된 파일임을 알 수 있음
# BB의 위치(Prediction)를 확인하여 darknet이 제대로 동작하는지 확인
imShow('predictions.jpg')

In [ ]:
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/dog.jpg

In [ ]:
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/eagle.jpg

In [ ]:
# class를 바꿔서 학습할 때마다 메뉴바의 predictions.jpg를 확인해야 함 : 하나의 코드가 실행될 때마다 덮어쓰기 되기 때문
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/giraffe.jpg

In [ ]:
!./darknet detector test cfg/coco.data cfg/yolov4.cfg yolov4.weights data/horses.jpg

# Custom Dataset 수집하기
 - Custom 이미지 데이터셋을 수집 후 라벨링 작업
 -  어느 정도 성능을 위해서 경험상 최소 500장 이상의 학습 이미지 데이터셋이 필요
 - Class 혹은 Label의 개수는 1개부터 많게는 수십 개도 가능.

# 이미지와 라벨링 파일 저장하기

- 전체 이미지 파일과 라벨링 파일을 모두 darknet/data/obj/ 경로에 넣어준다.
- obj 폴더가 darknet/data/ 경로 내에 없다면 생성 후 파일들을 집어 넣어준다.

In [ ]:
# https://drive.google.com/drive/folders/1ESsVvBu8LzAA12CRSChFqAlOleLKXN1_  
# download obj.zip, test.zip  : truck, bus, tank image
!cp /content/drive/MyDrive/data/obj.zip ../ # 현재 디렉토리(darknet)의 상위에 있는 content 디렉토리로 zip file 이동
!cp /content/drive/MyDrive/data/test.zip ../

In [ ]:
# data/obj, data/test 디렉토리 확인하면 이미지와 라벨링 정보가 동시에 들어있음
!unzip ../obj.zip -d data/obj/ # txt file : (class, x, y, h, w) BB Box의 위치 정보(이미지 속 트럭, 버스, 탱크의 개수만큼 저장되어 있음)
!unzip ../test.zip -d data/test/

In [ ]:
!pwd

# configure 파일 설정

 - darknet/cfg/yolov4-custom.cfg 복사 후 yolov4-obj.cfg 라는 이름으로 같은 경로 내에 복사본을 생성

 - darknet
 -  ㄴ cfg
 -    ㄴ yolov4-custom.cfg
 -    ㄴ yolov4-obj.cfg
 -    ㄴ ...
 -  ㄴ ...


# yolov4-obj.cfg 파일 수정  
- 3개의 class에 해당하는 이미지만 detection

  - 6번째 line을 batch=64로 수정
  - 7번째 line을 subdivision=16로 수정
  - 20번째 line을 max_batches={클래수 개수*2000}로 수정
    - 클래스의 개수가 2개라면 max_batches=4000
    - 클래스의 개수가 3개라면 max_batches=6000
    - 클래스의 개수가 5개라면 max_batches=10000
  - 22번째 line을 steps={클래스 개수*2000*0.80},{클래스 개수*2000*0.90}로 수정
    - 클래스 개수가 2개라면 steps=3200,3600
    - 클래스 개수가 3개라면 steps=4800,5400
    - 클래스 개수가 5개라면 steps=8000,9000
  - 8번째, 9번째 line을 원하는 네트워크 input size로 수정
    - 416x416 크기의 이미지를 입력하려면 width=416, height=416 (32의 배수로 설정하는 것을 권장)
  - [yolo] layer에 해당하는 970번째, 1058번째, 1146번째 3개의 line을 모두 classes={클래스 개수}로 수정
    - 클래스의 개수가 2개라면 classes=2
    - 클래스의 개수가 3개라면 classes=3
    - 클래스의 개수가 5개라면 classes=5
  - 3개의 [yolo] layer 직전의 [convolution] layer의 해당하는 963번째, 1051번째, 1139번째 line을 모두 filters={(클래수 개수 + 5) x 3}로 수정
    - 클래스의 개수가 2개라면 filters=21
    - 클래스의 개수가 3개라면 filters=24
    - 클래스의 개수가 5개라면 filters=30

# 데이터 파일 생성하기

- 이제 학습을 하기 위한 데이터들의 관한 정보를 담고 있는 obj.names, obj.data 파일 2개를 만드는 단계이다.

In [ ]:
# 80개가 아닌 3개의 class(bus, truck, tank)를 detect할 수 있도록 network 변경
# https://drive.google.com/file/d/1AVqPTug8JU1IZ4tvBu-ktWkHZOg8y-mf/view?usp=sharing에서 yolov4-obj.cfg 다운로드
!cp /content/drive/MyDrive/data/yolov4-obj.cfg ./cfg

- classes= 3
- train  = data/train.txt
- valid  = data/train.txt
- names = data/obj.names   # 3개의 classes가 어떤 이름을 갖고있는지 저장해놓은 file : 즉, 학습할 대상인 object의 이름
- backup = /content/drive/MyDrive/data/backup

In [ ]:
# https://drive.google.com/file/d/1aYG2iWG1RY1jzmP6_nfggnJfbLUnRO0a/view?usp=sharing 에서 obj.data 다운로드
# obj.data : 훈련된 file명이 data/train.txt에 저장되어 있고 data/obj.names에 3개의 classes가 무엇인지 저장되어 있음. 학습된 weight file은 backup 경로에 저장
!cp /content/drive/MyDrive/data/obj.data ./cfg

- truck
- bus
- tank

In [ ]:
# https://drive.google.com/file/d/1tM1-FAuksjw_8aXEiJmbmHJ4Pkuy2AQR/view?usp=sharing 에서 obj.names 다운로드
!cp /content/drive/MyDrive/data/obj.names ./data

# train.txt, valid.txt 파일 생성 및 저장하기
: train용, validation용 image 목록을 저장해야 하는데 자동화하여 python file로 저장해야 올바른 학습이 가능
- 이미지의 경로가 지정되어 있는 train.txt 파일과 test.txt 파일을 생성하고 저장하는 단계이다. 
- 각각의 파일은 학습과 검증에 사용될 이미지의 상대 혹은 절대 경로를 저장하고 있다.


## 1
- glob 라이브러리를 활용하여 경로를 이전에 저장한 이미지들의 경로를 리스트 형태로 생성한다. 아래의 코드에서 이미지 경로 설정에 주의한다.

- from glob import glob
- img_list = glob('darknet/build/darknet/x64/data/obj/*.jpg')
- len(img_list)
- 위 코드 실행 시 학습 및 검증의 사용될 전체 이미지의 개수가 출력된다.

## 2
- sklearn 라이브러리의 train_test_split 함수를 사용하여 학습과 검증에 사용할 이미지를 분할한다.

- from sklearn.model_selection import train_test_split
- train_img_list, test_img_list = train_test_split(img_list, test_size=0.1, random_state=42) # random_state는 임의로 선택
- print(len(train_img_list), len(test_img_list))

## 3
- 학습 이미지 리스트와 검증 이미지 리스트를 각각 train.txt와 test.txt 파일에 저장한다. 
- 각각의 txt 파일의 경로는 darknet/build/darknet/x64/data/ 경로 아래에 둔다. - 경로 설정에 주의한다.

- with open('darknet/build/darknet/x64/data/train.txt', 'w') as f:
-     f.write('\n'.join(train_img_list) + '\n')

- with open('darknet/build/darknet/x64/data/test.txt', 'w') as f:
-     f.write('\n'.join(test_img_list) + '\n')
- ex) 즉, train.txt와 test.txt의 파일의 내용은 예를 들어 아래와 같이 구성된다.

- darknet/build/darknet/x64/data/obj/img1.jpg
- darknet/build/darknet/x64/data/obj/img2.jpg
- darknet/build/darknet/x64/data/obj/img3.jpg
- ...


In [ ]:
# https://drive.google.com/file/d/1hGHmgYrRgAIk7-jTCxQTo3RCqzA2NOBF/view?usp=sharing 에서 generate_train.py, 
# https://drive.google.com/file/d/1sFjTcL6FM-dYner-l9AfI5nLom1iyyH9/view?usp=sharing 에서 generate_test.py 다운로드(나의 google drive의 data 디렉토리 아래에 저장하기)
!cp /content/drive/MyDrive/data/generate_train.py ./
!cp /content/drive/MyDrive/data/generate_test.py ./

In [ ]:
# 훈련할 file list 생성 : train.txt, test.txt 파일이 생성되었는지 확인
!python generate_train.py
!python generate_test.py

# 새로운 학습을 위해 필요한 yolov4.conv.137  pre-trained 가중치 파일을 다운로드

# Pre-trained Weight 파일 다운받기

 - 새로운 학습을 위해 필요한 yolov4.conv.137이라는 이름의 pre-trained 가중치 파일을 다운받는다.

 - 다운받은 yolov4.conv.137 파일은 darknet/build/darknet/ 경로에 둔다.
 - ex) /darknet/yolov4.conv.137

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

# 학습하기
- !./darknet detector train {obj.data 경로} {yolov4-obj.cfg 경로} {yolov4.conv.137 경로} -dont_show

- error message를 확인하지 않으려면 위와 같이 -dont_show 옵션을 입력하는 것이 좋다.
- -map 옵션을 입력하여 학습 그래프를 확인해볼 수도 있다.

In [ ]:
!./darknet detector train cfg/obj.data cfg/yolov4-obj.cfg yolov4.conv.137 -dont_show -map
# detector train, detector test
# cfg/obj.data : cfg/obj.names를 함께 끌고 온다
# cfg/yolov4-obj.cfg : 3개의 classes 학습할 수 있는 network file
# yolov4.conv.137 : pre-trained weight file
# 학습 monitoring : chart_yolov4-obj.png (학습이 몇 epoch 진행되었는지, loss값이 떨어지고 있는지 확인 가능)
# 1000 step마다 data 디렉토리 밑에 weight file, 1000 step별로 가장 cost가 낮은 best weight file, last weight file 생성됨
# 학습이 진행되어도 이전에 생성된 best weight file보다 cost가 낮아지지 않는다면 학습 효과가 없다는 뜻

# Test 하기

- 학습이 완료되면 테스트 진행

- !./darknet detector test {obj.data 경로} {yolov4-obj.cfg 경로} {학습된 weight 파일 경로} {테스트 이미지 파일 경로}

- 학습된 weight 파일은 obj.data에 backup으로 지정된 경로에 위치한다. 
- yolov4-obj_final.weights, yolov4-obj_best.weights, yolov4-obj_best.weights 파일이 생성된다.
- 코드 실행 결과로 predictions.jpg 파일이 생성된다.


In [ ]:
#!cp /content/drive/MyDrive/data/yolov4-obj_last.weights .
!cp /content/drive/MyDrive/data/yolov4-obj_best.weights .

Internet에서 tank bus truck을 다운로드 받는다.

In [ ]:
!./darknet detector test cfg/obj.data cfg/yolov4-obj.cfg yolov4-obj_last.weights ../tank01.jpg

In [ ]:
imShow('predictions.jpg')

In [ ]:
!./darknet detector test cfg/obj.data cfg/yolov4-obj.cfg yolov4-obj_last.weights ../truck01.jpg
imShow('predictions.jpg')

In [ ]:
!./darknet detector test cfg/obj.data cfg/yolov4-obj.cfg yolov4-obj_last.weights ../bus01.jpg
imShow('predictions.jpg')